# Lab in Data Science: Final Project

Pierre Fouche, Matthias Leroy and Raphaël Steinmann

## Imports

In [1]:
%matplotlib inline
import matplotlib.pylab as plt
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 18
plt.style.use('fivethirtyeight')

In [187]:
import getpass
import pyspark
from datetime import datetime
from pyspark.sql import SparkSession
import pyspark.sql.functions as functions
from pyspark.sql.types import BooleanType
from pyspark.sql.window import Window
import math
import helpers
import pickle
import numpy as np
import spark_helpers
import pandas as pd
import functools

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Initialize the `SparkSession`

In [3]:
conf = pyspark.conf.SparkConf()
conf.setMaster('yarn')
conf.setAppName('project-{0}'.format(getpass.getuser()))
conf.set('spark.executor.memory', '6g')
conf.set('spark.executor.instances', '6')
conf.set('spark.port.maxRetries', '100')
sc = pyspark.SparkContext.getOrCreate(conf)
conf = sc.getConf()
sc

<SparkContext master=yarn appName=project-fouche>

In [4]:
# init spark session
spark = SparkSession(sc)

## Loading the data

## Data Processing

### Cleaning metadata
First, let's clean the metadata dataframe:

In [5]:
# load metadata
raw_metadata = spark.read.load('/datasets/project/metadata', format='com.databricks.spark.csv', header='false', sep='\\t')

In [6]:
# remove multiple spaces
metadata = raw_metadata.withColumn('_c0', functions.regexp_replace(raw_metadata._c0, '\s+', ' '))
# split into columns
metadata = metadata.withColumn('name', functions.split(metadata._c0, '%')[1])
for (name, index, type_) in [('station_ID',0, 'int'), ('long',1, 'double'), ('lat',2, 'double'), ('height',3, 'int')]:
    metadata = metadata.withColumn(name, functions.split(metadata._c0, ' ')[index].cast(type_))
# remove useless column
metadata = metadata.drop('_c0')
# trim name column to remove left/right blank
metadata = metadata.withColumn('name', functions.trim(metadata.name))

In [7]:
metadata.show(5)

+----------------+----------+---------+---------+------+
|            name|station_ID|     long|      lat|height|
+----------------+----------+---------+---------+------+
|       Bucuresti|         2|26.074412| 44.44677|     0|
|          Calais|         3| 1.811446|50.901549|     0|
|      Canterbury|         4| 1.075329|51.284212|     0|
|          Exeter|         5|-3.543547|50.729172|     0|
|Fideris, Bahnhof|         7| 9.733756|46.922368|   744|
+----------------+----------+---------+---------+------+
only showing top 5 rows



We will use the SBB data limited around the Zurich area. We will focus on all the stops within 10km of the Zurich train station. Let's get rid of all the stations that are too far away from Zurich:

In [8]:
metadata.count()

25935

In [9]:
# coordinates of Zürich main train station
lat_zurich = 47.3782
long_zurich = 8.5402

In [10]:
# convert to pandas dataframe
pandas_df = metadata.toPandas()
# keep only the stops that are located < 10km from Zurich HB
pandas_df['distance_to_zh'] = pandas_df.apply(lambda x: helpers.distance(x['long'], x['lat'], long_zurich, lat_zurich), axis=1)
pandas_df = pandas_df[pandas_df['distance_to_zh'] < 10]

In [11]:
pandas_df

,name,station_ID,long,lat,height,distance_to_zh
74,Zimmerberg-Basistunnel,176,8.521961,47.351679,0,3.253243
2084,Urdorf,8502220,8.434713,47.390882,442,8.066052
2085,Birmensdorf ZH,8502221,8.437543,47.357432,488,8.068738
2086,Bonstetten-Wettswil,8502222,8.468175,47.325896,528,7.953967
2092,Urdorf Weihermatt,8502229,8.430330,47.380971,456,8.278324
2189,"Waldegg, Birmensdorferstrasse",8502559,8.463472,47.368305,588,5.881706
2202,"Zürich, Goldbrunnenplatz",8502572,8.513918,47.370293,421,2.165596
2320,"Aesch ZH, Gemeindehaus",8502876,8.438705,47.338209,537,8.844291
2329,"Bonstetten, Dorfplatz",8502885,8.467781,47.315088,528,8.889213
2344,"Birmensdorf ZH, Zentrum",8502950,8.437173,47.353936,468,8.215029


In [12]:
# pandas_df.distance_to_zh.max()

In [13]:
# recreate spark dataframe from pandas dataframe
metadata = spark.createDataFrame(pandas_df)
# create dict of stations from pandas dataframe
stations = pandas_df.set_index('station_ID').to_dict('index')

### Cleaning main dataset

In [14]:
# load full data
raw_df = spark.read.load('/datasets/project/istdaten/*/*', format='csv', header='true', inferSchema='true', sep=';')
# load sample data
#raw_df = spark.read.load('/datasets/project/istdaten/2018/01', format='csv', header='true', inferSchema='true', sep=';')

In [15]:
# rename the fields german -> english
fields = {
    'BETRIEBSTAG':'date',
    'FAHRT_BEZEICHNER':'trip_id',
    'PRODUKT_ID':'transport_type',
    'LINIEN_ID':'train_id',
    'VERKEHRSMITTEL_TEXT':'train_type',
    'ZUSATZFAHRT_TF':'additional_trip',
    'FAELLT_AUS_TF':'trip_failed',
    'HALTESTELLEN_NAME':'stop_name',
    'BPUIC':'stop_id',
    'ANKUNFTSZEIT':'schedule_arrival',
    'AN_PROGNOSE':'real_arrival',
    'AN_PROGNOSE_STATUS':'arr_forecast_status',
    'ABFAHRTSZEIT':'schedule_dep',
    'AB_PROGNOSE':'real_dep',
    'AB_PROGNOSE_STATUS':'dep_forecast_status',
    'DURCHFAHRT_TF':'no_stop_here'
}

df = raw_df.selectExpr([k + ' as ' + fields[k] for k in fields])

In [16]:
# refactor dates
df = df.withColumn('date', functions.from_unixtime(functions.unix_timestamp('date', 'dd.MM.yyyy')))
df = df.withColumn('schedule_arrival', functions.from_unixtime(functions.unix_timestamp('schedule_arrival', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('real_arrival', functions.from_unixtime(functions.unix_timestamp('real_arrival', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('schedule_dep', functions.from_unixtime(functions.unix_timestamp('schedule_dep', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('real_dep', functions.from_unixtime(functions.unix_timestamp('real_dep', 'dd.MM.yyyy HH:mm')))

In [17]:
# add a column containing the weekday (monday=1, sunday=6)
df = df.withColumn('weekday', spark_helpers.get_weekday(df.date))

In [18]:
# keep only the rows with stops near zurich
df = df.where(df.stop_id.isin([int(x) for x in list(pandas_df.station_ID.unique())]))

In [19]:
# there is still 51'571'541 rows in zurich area
# df.count()

In [20]:
# keep only date after the 10th of december, because the schedule changed
df = df.where(df.date > '2017-12-10 00:00:00')

In [21]:
# discard the rows when there is no stop here
df2 = df.where(df.no_stop_here == 'false')

In [22]:
# discard ill-formated rows where the train leaves a station before arriving in it
df2 = df2.where((df2.schedule_dep >= df2.schedule_arrival) | functions.col('schedule_arrival').isNull() | functions.col('schedule_dep').isNull())

## Modeling the network

### From stops to trips

In [23]:
# create a column with the schedule time that will be used to build the network
df2 = df2.withColumn('schedule_time', spark_helpers.date_choice(df2.schedule_arrival, df2.schedule_dep))
#df2 = df2.withColumn('schedule_time', functions.from_unixtime(functions.unix_timestamp('schedule_time', 'dd.MM.yyyy HH:mm')))

# create a column that tells if a stop is the first/last one of its trip or in the middle
df2 = df2.withColumn('stop_type', spark_helpers.stop_type(df2.schedule_dep, df2.schedule_arrival))

In [24]:
trips = df2.select(['trip_id', 'date', 'schedule_time', 'stop_id', 'stop_type', 'schedule_arrival', 'schedule_dep', 'transport_type', 'train_type', 'arr_forecast_status', 'weekday', 'real_arrival']).orderBy(['trip_id', 'schedule_time'], ascending=[0,0])

In [25]:
# duplicate the dataframe, shift the copy of one row and append it to the original
# this way, we have for each row the current stop and the next stop
w = Window().partitionBy(functions.col('trip_id')).orderBy(functions.col('trip_id'))
trips2 = trips.select("*", functions.lag("trip_id").over(w).alias("next_tid"))
trips2 = trips2.select("*", functions.lag("schedule_time").over(w).alias("next_time"))
trips2 = trips2.select("*", functions.lag("stop_id").over(w).alias("next_sid"))
trips2 = trips2.select("*", functions.lag("stop_type").over(w).alias("next_type"))
trips2 = trips2.select("*", functions.lag("schedule_arrival").over(w).alias("next_sched_arr"))
trips2 = trips2.select("*", functions.lag("schedule_dep").over(w).alias("next_sched_dep"))
trips2 = trips2.select("*", functions.lag("arr_forecast_status").over(w).alias("next_arr_forecast_status"))
trips2 = trips2.select("*", functions.lag("real_arrival").over(w).alias("next_real_arrival"))

trips2 = trips2.where(trips2.next_time.isNotNull())

In [26]:
#trips2.where(trips2.stop_type=='first').count()
#trips2.where(trips2.stop_type=='last').count()
#trips2.where(trips2.stop_type=='mid').count()

In [27]:
# create a new column telling if the edge is valid or not
# (i.e. if the stop and next stop are really part of the same ride)
trips3 = trips2.withColumn('is_valid', spark_helpers.edge_is_valid(trips2.trip_id, trips2.schedule_time, trips2.stop_id, trips2.stop_type, trips2.next_tid, trips2.next_time, trips2.next_sid, trips2.next_type, trips2.schedule_dep,trips2.next_sched_arr))

In [28]:
# keep only valid edges
trips4 = trips3.filter(trips3.is_valid=='true')

In [29]:
# trips4.select('stop_id', 'next_sid').distinct().count()
# gives 6606

# trips3.select('stop_id', 'next_sid').distinct().count()
# gives 8557

## Prediction / Regression

Deux idées : Le but c'est de prévoir le retard ou l'avance d'un départ ou d'une arrivée pour un arrêt précis à un moment précis et un trip précis.

- Faire une vraie regression ou n'importe quel algo pour déterminer le retard. On utilise comme features : l'arrêt (catégorie), latitude longitude (peut être mettre ensemble en tuple), jour precis ou jour de la semaine ?, le type de transport (bus-train ...), le trip id ou redondant ???, 

- Faire une moyenne, à un arrêt, un trip, un jour, une heure la moyenne de retard qu'il a eu dans ses conditions ... 

On va avoir besoin des trips id dans le dico histoire de pouvoir lier le graph des arrêts avec le réél trajet, quel bus/train fait cet edge.

## Predictive model :

In this part we are going to build a preditive model using historical arrival/departure time data. Thus, our goal is to predict an uncertainty or certainty rate of taking a transport change with success, and finally to spread it for a whole journey using a route planning algorithm explained in the next section.

### 1) Processing the datas :

First of all, we have to group the data that match (same trip between two transport stops at the same schedule), and to compute the delay for the trips where the historical time was measured.

In [31]:
# We drop what we do not need for the prediction
trips5 = trips4.drop('stop_type', 'next_type', 'is_valid', 'arr_forcast_status', 
                     'schedule_time', 'schedule_arrival', 'next_sched_dep', 
                     'next_time', 'next_tid', 'real_arrival', 'arr_forecast_status')

# We refactor the dates by keeping only the time of the day and not the entire day
trips5 = trips5.withColumn('schedule_dep', spark_helpers.keep_time(trips5.schedule_dep))
trips5 = trips5.withColumn('next_sched_arr', spark_helpers.keep_time(trips5.next_sched_arr))
trips5 = trips5.withColumn('next_real_arrival', spark_helpers.keep_time(trips5.next_real_arrival))

# We compute the arrival delay of each stop station
df3 = trips5.withColumn("delay_arrival", 
                     functions.unix_timestamp('next_real_arrival', 'HH:mm:ss') -
                     functions.unix_timestamp('next_sched_arr', 'HH:mm:ss'))

# We create time interval in order to differentiate rush hour and other time of the day
# We have to try different combinations of interval time (that gave best prediction)
df3 = df3.withColumn('arrival_interval', spark_helpers.create_rush(df3.next_sched_arr))
# We also gather the week day in 4 buckets (Wednesnay, Saturday and Sunday separatly and the other days together)
df3 = df3.withColumn('weekday', spark_helpers.group_weekday(df3.next_sched_arr))

# We create a dataframe where we only keep the rows where there is a real arrival time
arrival_df = df3.filter(df3.next_arr_forecast_status == "GESCHAETZT")
arrival_df = arrival_df.drop('next_arr_forecast_status')

arrival_df.groupby('transport_type').count().show()

+--------------+-------+
|transport_type|  count|
+--------------+-------+
|           Zug|1385934|
+--------------+-------+



As we can see only the trains are concerned with historical time data, thus we were able to compute the delay only for those trips. Moreover we only had 1 millions rows of evaluated time on the 30 millions trips that we have. 

So, one solution was to consider that there is no delay for the other trips (Bus / Tram), thus, there will almost alway have 0% chance to miss a connection between two transports as there are a majority of Bus and Tram in the center of Zurich.

An other solution was to try to predict the delay of those trips according to the ones we already had. That is why we decided to make a linear regression using the localization (latitude, longitude) of the stop, as along with the day and time of the trip.

### 2) Complete the delay data with Linear Regression :

# MATTHIAS A COMPLETER

### 3) Evaluate the uncertainty according to the delay:

Thus for now we have the delay for a majority of trips for a weekday and a time interval. Thus we have to find a way to evaluate the uncertainty according to the delays we have. We decide to use the interquartile range (IQR) in order to manage the outliers. In fact, the interquartile range is the difference between the upper and lower quartiles (75th, 25th percentiles) Q3 and Q2. 

$IQR = Q3 - Q2$

It allows us to mesure the dispersion of the delays for each trip. Moreover we associate it with the interquartile mean (IQM) which is the mean of the data included in the interquartile range and is insensitive to outliers.

$IQM = \frac{2}{n}\sum_{i = \frac{n}{4}+1}^\frac{3n}{4} x_i$

Thus for each trips, for a given day of the week and time interval, we now have an estimation of the worst delay possible without being affected by the outliers.

In [33]:
# We group the trips by their week day and time interval
delay_distribution = arrival_df.groupby(['trip_id', 'arrival_interval', 'weekday']).agg(functions.collect_list(functions.col('delay_arrival'))).alias('distri')
delay_distribution = delay_distribution.withColumn('distri', spark_helpers.delete_neg(functions.col('collect_list(delay_arrival)')))

# We compute the IQM / IQR and worst case
delay_distribution = delay_distribution.withColumn('IQM', spark_helpers.iqm(delay_distribution.distri))
delay_distribution = delay_distribution.withColumn('interquartile', spark_helpers.interquartile(delay_distribution.distri))
delay_distribution = delay_distribution.withColumn('worst_case', delay_distribution.IQM + delay_distribution.interquartile)

#delay_distribution_pd = delay_distribution.toPandas()

# Then we store it in order to easily make our predictions.
regenerate_distri_pickle = False
if regenerate_distri_pickle:
    delay_distribution_pd.to_pickle('./data/delay_distri.pickle')
else:
    delay_distribution_pd = pd.read_pickle('./data/delay_distri.pickle')

delay_distribution_pd

,trip_id,arrival_interval,weekday,collect_list(delay_arrival),distri,IQM,interquartile,worst_case
0,85:11:18231:002,0,0,"[0, -60, 0, 60, 0, 0, -60, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 60, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",19.636363636363637,60.0,79.636364
1,85:11:70828:002,0,0,[-180],[0],0.0,0.0,0.000000
2,85:11:88764:003,0,0,"[0, -60, 60, 0, 0, 0]","[0, 0, 60, 0, 0, 0]",0.0,0.0,0.000000
3,85:11:88853:005,1,0,"[60, 0, 60, 0, -60, 0, -60, 0, 0]","[60, 0, 60, 0, 0, 0, 0, 0, 0]",0.0,0.0,0.000000
4,85:11:19536:001,1,0,"[0, 0, 0, 0, 0, 0, 0, -60, 0, 0, 0, 0, -60, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0,0.0,0.000000
5,85:11:18330:001,1,0,"[60, 0, 0, 0, 60, 0, -60, 0, 120, 60, 0, 60, 6...","[60, 0, 0, 0, 60, 0, 0, 0, 120, 60, 0, 60, 60,...",24.485294117647058,60.0,84.485294
6,85:11:31432:003,1,0,"[-180, 60, 0, 0, 0, -120, 60, 0, 0, 0]","[0, 60, 0, 0, 0, 0, 60, 0, 0, 0]",0.0,0.0,0.000000
7,85:11:2684:001,1,0,"[-60, -60, 120, 60, -60, 0, -60, -60, 0, 0, -6...","[0, 0, 120, 60, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4.285714285714286,60.0,64.285714
8,85:11:70727:008,0,0,[-60],[0],0.0,0.0,0.000000
9,85:11:89538:000,1,0,"[240, 240, 240, 240]","[240, 240, 240, 240]",240.0,0.0,240.000000


In [49]:
delay_distribution_pd[delay_distribution_pd.trip_id == '85:11:2648:001']

,trip_id,arrival_interval,weekday,collect_list(delay_arrival),distri,IQM,interquartile,worst_case
1214,85:11:2648:001,1,0,"[60, 60, -60, 60, 60, 60, 0, 0, -60, -60, -60,...","[60, 60, 0, 60, 60, 60, 0, 0, 0, 0, 0, 0, 60, ...",8.175182481751825,60.0,68.175182


Thus, we can now use those values in order to predict a % of missing a connection when we are in a trip, a certain day and at a certain time.

# ADD RAPH PART

### Determining uncertainty of a trip according to the predictive model



In [216]:
def search_inter(trip_id, weekday, time_interval):
    interquartile = delay_distribution_pd[(delay_distribution_pd.trip_id == str(trip_id)) & 
                            (delay_distribution_pd.arrival_interval == str(time_interval)) & 
                            (delay_distribution_pd.weekday == str(weekday))].worst_case
    #print(interquartile)
    return float(interquartile) if interquartile.size != 0 else float(0)

def rush_inter(time):
    if (time >= '06:00:00' and time <= '09:00:00') or (time >= '17:00:00' and time <= '19:00:00'):
        return 0
    else:
        return 1    

def routing_algo(path):
    prev_edge = path[0]
    certainty = {}
    for i, edge in enumerate(path):
        
        if (edge[4] != prev_edge[4]) and (edge[4] != 'walk') and ((prev_edge[4] != 'walk') or (i-1 != 0)):
            
            time_for_change = edge[2] - prev_edge[3]
            
            if (prev_edge[4] == 'walk') and (i-1 != 0):      
                time_inter = spark_helpers.rush_inter(path[i-2][3].time().isoformat())
                inter = search_inter(path[i-2][4], spark_helpers.group_weekday_py(path[i-2][3].weekday()), time_inter)
            else:
                time_inter = spark_helpers.rush_inter(prev_edge[3].time().isoformat())
                inter = search_inter(prev_edge[4], spark_helpers.group_weekday_py(edge[3].weekday()), time_inter)
            
            #print(time_for_change, time_inter, inter)
            
            if inter:
                certainty[i-1] = min(time_for_change.total_seconds() / inter, 1)
            else:
                certainty[i-1] = 1
        prev_edge = edge
        
        if certainty:
            certainty_tot = functools.reduce((lambda v1,v2: v1*v2), certainty.values())
        else:
            certainty_tot = 1
            
    return certainty, certainty_tot

In [217]:
for j,i in enumerate(shortest_paths):
    print(j, routing_algo(i))

0 ({1: 1, 6: 1}, 1)
1 ({1: 0.5602627082212257, 7: 1}, 0.5602627082212257)
2 ({6: 1}, 1)
3 ({}, 1)
4 ({}, 1)
5 ({0: 1}, 1)
6 ({1: 1, 3: 1}, 1)
7 ({0: 1, 1: 1, 4: 1}, 1)
8 ({1: 1}, 1)
9 ({2: 1}, 1)
10 ({2: 1}, 1)
11 ({0: 0.8800856531049251}, 0.8800856531049251)
12 ({1: 1, 2: 1}, 1)
13 ({0: 1, 1: 1, 4: 1}, 1)
14 ({}, 1)
15 ({0: 1}, 1)
16 ({3: 0.24631029588938713}, 0.24631029588938713)
17 ({0: 1}, 1)
18 ({}, 1)
19 ({1: 0.5602627082212257, 7: 1}, 0.5602627082212257)
20 ({1: 0.5602627082212257, 7: 1, 12: 1, 16: 1}, 0.5602627082212257)
21 ({6: 1, 10: 1, 13: 0.10165393694406753}, 0.10165393694406753)
22 ({1: 0.34653073361884373, 2: 1, 3: 1}, 0.34653073361884373)
23 ({3: 1}, 1)
24 ({1: 1}, 1)
25 ({0: 1}, 1)
26 ({2: 1}, 1)
27 ({1: 0.34653073361884373}, 0.34653073361884373)
28 ({}, 1)
29 ({0: 1, 2: 0.9158410148514851}, 0.9158410148514851)
30 ({0: 0.8800856531049251}, 0.8800856531049251)
31 ({0: 1}, 1)
32 ({1: 0.5602627082212257, 3: 1}, 0.5602627082212257)
33 ({0: 1}, 1)
34 ({1: 0.3465307336188437

332 ({1: 1, 6: 1}, 1)
333 ({0: 1}, 1)
334 ({}, 1)
335 ({1: 0.34653073361884373, 2: 1}, 0.34653073361884373)
336 ({1: 1, 6: 1}, 1)
337 ({0: 1}, 1)
338 ({}, 1)
339 ({2: 1, 3: 1}, 1)
340 ({1: 1, 2: 1}, 1)
341 ({1: 1}, 1)
342 ({0: 1}, 1)
343 ({0: 1}, 1)
344 ({}, 1)
345 ({0: 1}, 1)
346 ({6: 1}, 1)
347 ({2: 1}, 1)
348 ({}, 1)
349 ({0: 1}, 1)
350 ({6: 1}, 1)
351 ({}, 1)
352 ({}, 1)
353 ({0: 1}, 1)
354 ({}, 1)
355 ({6: 1, 9: 1}, 1)
356 ({1: 0.34653073361884373, 2: 1}, 0.34653073361884373)
357 ({1: 1, 6: 1, 9: 1}, 1)
358 ({1: 0.5602627082212257, 7: 1}, 0.5602627082212257)
359 ({2: 1, 4: 1}, 1)
360 ({0: 1}, 1)
361 ({1: 0.5602627082212257, 7: 1, 12: 1}, 0.5602627082212257)
362 ({1: 1, 6: 1, 9: 1}, 1)
363 ({1: 0.34653073361884373, 2: 1}, 0.34653073361884373)
364 ({0: 1}, 1)
365 ({1: 1}, 1)
366 ({3: 1}, 1)
367 ({0: 1, 1: 1, 4: 1}, 1)
368 ({1: 0.5602627082212257, 7: 1}, 0.5602627082212257)
369 ({0: 1}, 1)
370 ({}, 1)
371 ({1: 1, 4: 1}, 1)
372 ({3: 0.24631029588938713}, 0.24631029588938713)
373 ({0: 

In [221]:
shortest_paths[13]

[(8503000,
  8503020,
  datetime.datetime(2018, 1, 16, 14, 1),
  datetime.datetime(2018, 1, 16, 14, 3),
  '85:11:18652:001',
  'S6'),
 (8503020,
  8503001,
  datetime.datetime(2018, 1, 16, 14, 11),
  datetime.datetime(2018, 1, 16, 14, 14),
  '85:11:18552:001',
  'S5'),
 (8503001,
  8503509,
  datetime.datetime(2018, 1, 16, 14, 21),
  datetime.datetime(2018, 1, 16, 14, 23),
  '85:11:19252:001',
  'S12'),
 (8503509,
  8590790,
  datetime.datetime(2018, 1, 16, 14, 23),
  datetime.datetime(2018, 1, 16, 14, 28, 39, 359227),
  'walk',
  'walk'),
 (8590790,
  8590802,
  datetime.datetime(2018, 1, 16, 14, 28, 39, 359227),
  datetime.datetime(2018, 1, 16, 14, 31, 33, 747405),
  'walk',
  'walk'),
 (8590802,
  8593529,
  datetime.datetime(2018, 1, 16, 14, 33),
  datetime.datetime(2018, 1, 16, 14, 33),
  '85:849:301997-32301-1',
  '302'),
 (8593529,
  8590832,
  datetime.datetime(2018, 1, 16, 14, 34),
  datetime.datetime(2018, 1, 16, 14, 34),
  '85:849:301997-32301-1',
  '302'),
 (8590832,
  8594

### For each day of the week, model the network
Get the edges of the network and the departure/arrival times for each trip (edge=trip)
We assume the schedule repeat every week, and we generate one schedule per weekday.
Days off have the same schedules as sundays.

In [35]:
# creating a model for each day of the week
# this code needs to be run only once
typical_monday = '2018-01-15 00:00:00'
typical_tuesday = '2018-01-16 00:00:00'
typical_wednesday = '2018-01-17 00:00:00'
typical_thursday = '2018-01-18 00:00:00'
typical_friday = '2018-01-19 00:00:00'
typical_saturday = '2018-01-20 00:00:00'
typical_sunday = '2018-01-21 00:00:00'
typical_week = [typical_monday,typical_tuesday,typical_wednesday,typical_thursday,typical_friday,typical_saturday,typical_sunday]

In [36]:
regenerate_models = False
days_names = ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']

# generate one network for each weekday and store them in pickles
if regenerate_models:
    for (date, day_name) in zip(typical_week, days_names):
        network = (helpers.model_network(trips4, date))
        with open('./data/'+day_name+'.pickle', 'wb') as handle:
            helpers.network_to_datetime(network) # works inplace
            pickle.dump(network, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(str(day_name) + ' done')

In [54]:
# load the networks from the pickles
models = []
for day in days_names:
    with open('./data/'+ day +'.pickle', 'rb') as handle:
        network = pickle.load(handle)
    models.append(network)
    print(day + ' loaded')

ZH_HB_ID = 8503000

# compute walking network
walking_network = helpers.compute_walking_network(stations)
print('walking network loaded')


# example
sp = helpers.shortest_path(models, walking_network, stations,ZH_HB_ID, 8590694, datetime(2018, 1, 15, 14, 24))
helpers.reduced_path_tostring(helpers.reduce_path(sp), stations)

monday loaded
tuesday loaded
wednesday loaded
thursday loaded
friday loaded
saturday loaded
sunday loaded
walking network loaded
line S8 from Zürich HB to Zürich Oerlikon 14:25 -> 14:29(2 stops)
line S15 from Zürich Oerlikon to Glattbrugg 14:30 -> 14:32(1 stops)
line walk from Glattbrugg to Glattbrugg, Post 14:32 -> 14:34(1 stops)
line 768 from Glattbrugg, Post to Zürich Flughafen, Werft 14:36 -> 14:41(4 stops)
line walk from Zürich Flughafen, Werft to Kloten, Wallisellenstrasse 14:41 -> 14:59(4 stops)


In [38]:
import random
# select two stops at random and create a shortest path. Do that 100 times
shortest_paths = []
n_runs = 500
date = datetime(2018, 1, 16, 14)
source = ZH_HB_ID # zurich HB
reachable_stations_ids = helpers.get_reachable_stations(models[date.weekday()], walking_network, source)
reachable_stations = {sid: stations[sid] for sid in reachable_stations_ids}
for i in range(n_runs):
    if i%10==0:
        print(100*i/n_runs, '% finished')
    #source = random.choice(list(stations.keys()))
    dest = random.choice(list(reachable_stations.keys()))
    shortest_paths.append(helpers.shortest_path(models, walking_network, reachable_stations, source, dest, date))

0.0 % finished
2.0 % finished
4.0 % finished
6.0 % finished
8.0 % finished
10.0 % finished
12.0 % finished
14.0 % finished
16.0 % finished
18.0 % finished
20.0 % finished
22.0 % finished
24.0 % finished
26.0 % finished
28.0 % finished
30.0 % finished
32.0 % finished
34.0 % finished
36.0 % finished
38.0 % finished
40.0 % finished
42.0 % finished
44.0 % finished
46.0 % finished
48.0 % finished
50.0 % finished
52.0 % finished
54.0 % finished
56.0 % finished
58.0 % finished
60.0 % finished
62.0 % finished
64.0 % finished
66.0 % finished
68.0 % finished
70.0 % finished
72.0 % finished
74.0 % finished
76.0 % finished
78.0 % finished
80.0 % finished
82.0 % finished
84.0 % finished
86.0 % finished
88.0 % finished
90.0 % finished
92.0 % finished
94.0 % finished
96.0 % finished
98.0 % finished


In [ ]:
for sp in shortest_paths:
    print(routing_algo(sp))

# Visu map for checking

In [53]:
pandas_df.head()

,name,station_ID,long,lat,height,distance_to_zh
74,Zimmerberg-Basistunnel,176,8.521961,47.351679,0,3.253243
2084,Urdorf,8502220,8.434713,47.390882,442,8.066052
2085,Birmensdorf ZH,8502221,8.437543,47.357432,488,8.068738
2086,Bonstetten-Wettswil,8502222,8.468175,47.325896,528,7.953967
2092,Urdorf Weihermatt,8502229,8.430330,47.380971,456,8.278324


In [222]:
# Simply using the provided link
from pyproj import Proj, transform
def to_merc(lat, long):
    return transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), long, lat)

In [223]:
def create_line(a, index):
    result = []
    if len(index) == 1:
        result.append(a[:index[0]+1])
        result.append(a[index[0]:])
    else:
        for i,j in enumerate(index):
            if i+1 == len(index):
                result.append(a[index[i-1]:j+1])
                result.append(a[j:])
            elif i == 0:
                result.append(a[:j+1])
            else:
                result.append(a[index[i-1]:j+1])
    return result

In [225]:

def create_data_for_visu(pandas_df, path):
    x = []
    y = []
    transport = []
    labels = []
    chang = []

    for i, road in enumerate(path):
        long1 = pandas_df[pandas_df.station_ID == road[0]].long.values[0]
        long2 = pandas_df[pandas_df.station_ID == road[1]].long.values[0]

        lat1 = pandas_df[pandas_df.station_ID == road[0]].lat.values[0]
        lat2 = pandas_df[pandas_df.station_ID == road[1]].lat.values[0]

        lab1 = pandas_df[pandas_df.station_ID == road[0]].name.values[0]
        lab2 = pandas_df[pandas_df.station_ID == road[1]].name.values[0]

        merc1 = to_merc(lat1, long1)
        merc2 = to_merc(lat2,long2)

        x.append(merc1[0])
        x.append(merc2[0])

        y.append(merc1[1])
        y.append(merc2[1])

        labels.append(lab1)
        labels.append(lab2)

        transport.append(road[5])

        if transport[i-1] != transport[i]:
            chang.append(i)


    labels = list(dict.fromkeys(labels))
    x = list(dict.fromkeys(x))
    y = list(dict.fromkeys(y))

    lines_x = create_line(x,chang)
    lines_y = create_line(y,chang)

    labels = ['' if (i not in chang) and (i!=0) and (i!=len(labels)-1) else lab for i,lab in enumerate(labels)]

    transport_array = []
    transport_array.append(transport[0])
    [transport_array.append(t) for i,t in enumerate(transport[1:]) if t != transport[i]]

    colormap = {t:Set3[12][i] for i,t in enumerate(set(transport))}
    colors = [colormap[x] for x in transport_array]

    return labels, x, y, lines_x, lines_y, transport_array, colors

In [228]:
from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.models.markers import Square
from bokeh.models.glyphs import Patches
from bokeh.palettes import Set3
import time

def plot_trip(pandas_df, path):
    output_notebook()

    labels, x, y, lines_x, lines_y, transport_array, colors = create_data_for_visu(pandas_df, path)

    source = ColumnDataSource(data=dict(x=x, y=y, names=labels))

    label_set = LabelSet(x='x', y='y', text='names', level='glyph',
                  x_offset=-10, y_offset=10, source=source, render_mode='canvas')

    zurich_coord = to_merc(47.395501, 8.502376)
    x_y_offset = 6000

    p = figure(x_range=(-x_y_offset+zurich_coord[0], x_y_offset+zurich_coord[0]), 
               y_range=(-x_y_offset+zurich_coord[1], x_y_offset+zurich_coord[1]),
               x_axis_type="mercator", 
               y_axis_type="mercator")

    p.add_tile(CARTODBPOSITRON)

    for (colr, leg, x, y ) in zip(colors, transport_array, lines_x, lines_y):
        my_plot = p.line(x, y, color = colr, legend = leg, line_width=4)

    #p.multi_line(xs=lines_x, ys=lines_y, color=colors, line_width=4, legend=transport)
    p.circle(x="x", y="y", size=10, fill_color="#000000", source=source)

    p.add_layout(label_set)

    show(p)

In [229]:
plot_trip(pandas_df, shortest_paths[2])

Loading BokehJS ...

In [590]:
output_notebook()

source = ColumnDataSource(data=dict(x=x, y=y, names=labels))

label_set = LabelSet(x='x', y='y', text='names', level='glyph',
              x_offset=-10, y_offset=10, source=source, render_mode='canvas')

zurich_coord = to_merc(47.395501, 8.502376)
x_y_offset = 6000

p = figure(x_range=(-x_y_offset+zurich_coord[0], x_y_offset+zurich_coord[0]), 
           y_range=(-x_y_offset+zurich_coord[1], x_y_offset+zurich_coord[1]),
           x_axis_type="mercator", 
           y_axis_type="mercator")

p.add_tile(CARTODBPOSITRON)

p.circle(x="x", y="y", size=100, fill_color="#000000", source=source, fill_alpha=0.5, line_width = 0.01)

p.add_layout(label_set)

show(p)

Loading BokehJS ...

## PREDICTION / pandas and pyspark

In [ ]:
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import pandas as pd

In [ ]:
le_train_type = preprocessing.LabelEncoder()
le_train_type.fit(arrival_pd_df['train_type'])
# see the class
#list(le_train_type.classes_)

In [ ]:
arrival_pd_df['train_type_cat'] = le_train_type.transform(arrival_pd_df['train_type'])
# we can do the inverse
# arrival_pd_df['train_type'] = le_train_type.inverse_transform(arrival_pd_df['train_type_cat'])

In [ ]:
arrival_pd_df

In [ ]:
arrival_pd_df.weekday = arrival_pd_df.weekday.astype(int)
arrival_pd_df.arrival_interval = arrival_pd_df.arrival_interval.astype(int)
arrival_pd_df.arrival_interval = arrival_pd_df.arrival_interval.astype(int)

arrival_pd_df.dtypes

In [ ]:
week_df = arrival_pd_df[arrival_pd_df.weekday == 0]
week_df.delay_arrival = week_df.delay_arrival/60
week_df

In [ ]:
lin_regr = LinearRegression()
lin_regr.fit(week_df[['stop_id', 'next_sid', 'arrival_interval', 'train_type_cat']], week_df.delay_arrival)
pred = lin_regr.predict(week_df[['stop_id', 'next_sid', 'arrival_interval', 'train_type_cat']])
lin_regr.score(week_df[['stop_id', 'next_sid', 'arrival_interval', 'train_type_cat']],  week_df.delay_arrival)

In [ ]:
week_df['pred'] = pred
week_df

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

trip_Indexer = StringIndexer(inputCol="trip_id", outputCol="trip_id_index")
trip_model = trip_Indexer.fit(arrival_df)

arrival_df = trip_model.transform(arrival_df)

In [ ]:
arrival_df = arrival_df.drop('trip_id', 'transport_type')

In [ ]:
arrival_df = arrival_df.withColumn('weekday', arrival_df['weekday'].cast('int'))
arrival_df = arrival_df.withColumn('arrival_interval', arrival_df['arrival_interval'].cast('int'))

In [ ]:
arrival_df = arrival_df.withColumn('delay_arrival', arrival_df['delay_arrival'] / 60)

In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler


vec_ass = VectorAssembler(inputCols=['stop_id','weekday','arrival_interval','trip_id_index','transport_type_index'], outputCol="features")

#arrival_df = vec_ass.transform(arrival_df)
lr = LinearRegression(maxIter=5, regParam=0.0, solver="normal", 
                      labelCol='delay_arrival')
model = lr.fit(arrival_df)

test = model.transform(arrival_df)

In [ ]:
test = test.withColumn('evaluation', test['delay_arrival'] - test['prediction'])

In [ ]:
test.describe('evaluation').show()

In [ ]:
test.describe('delay_arrival').show()

In [ ]:
test.describe('prediction').show()

In [ ]:
test_df = arrival_df.limit(50).toPandas()
test_df

## Time Buckets

It was an attempt of finding time buckets directly according to the delay.
However Kmeans does not return meaningful clusters. That is why we decide to create our own interval based on main daily period. 

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder, StringIndexer

train_type_Indexer = StringIndexer(inputCol="train_type", outputCol="train_type_cat")
train_type_model = train_type_Indexer.fit(arrival_df)
arrival_df = train_type_model.transform(arrival_df)

arrival_df = arrival_df.withColumn('weekday', arrival_df['weekday'].cast('int'))
arrival_df = arrival_df.withColumn('train_type_cat', arrival_df['train_type_cat'].cast('int'))

arrival_df_test = arrival_df.filter(arrival_df.weekday < 5)
arrival_df_test = arrival_df_test.withColumn('delay_arrival', arrival_df_test.delay_arrival/60)

vec_ass = VectorAssembler(inputCols=['delay_arrival', 'train_type_cat'], outputCol="features")
kmeans_df = vec_ass.transform(arrival_df_test)
kmean = KMeans(k=7)
model_km = kmean.fit(kmeans_df)
kmeans_df = model_km.transform(kmeans_df)

In [ ]:
kmeans_df = kmeans_df.withColumn('timestamp', functions.unix_timestamp('next_sched_arr',  'HH:mm:ss'))

In [ ]:
mean_ = kmeans_df.groupBy('prediction').mean('timestamp').collect()
print(mean_)

In [ ]:
for i in mean_:
    print(i['prediction'])
    print(datetime.datetime.fromtimestamp(i['avg(timestamp)']).strftime("%H:%M:%S"))